In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
from IPython.display import clear_output
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')

In [2]:
def get_time_spaced_traindata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    return data

def get_time_spaced_testdata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1  
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    aq_data = data['PM25_Concentration']
    data.drop(columns=['PM25_Concentration'])
    return data,aq_data

In [30]:
fold=2 # not using for loop to avoid ram overflow
print('fold', fold)

train_data = pd.read_csv('../data/NP/fold_'+str(fold)+'_train_nsgp.csv.gz')
train_data['time'] = pd.to_datetime(train_data['time'])
train_data = train_data.set_index('time').sort_values(['time', 'station_id'])

test_data = pd.read_csv('../data/NP/fold_'+str(fold)+'_test_nsgp.csv.gz')
test_data['time'] = pd.to_datetime(test_data['time'])
test_data = test_data.set_index('time').sort_values(['time', 'station_id'])

fold 2


In [31]:
train_input = get_time_spaced_traindata(train_data)
test_input,test_output = get_time_spaced_testdata(test_data)

In [32]:
display(train_input,test_input,test_output)

,station_id,longitude,latitude,PM25_Concentration,weather,temperature,humidity,wind_speed,wind_direction,filled,delta_t
time,,,,,,,,,,,
2014-05-01 00:00:00,1002,116.205310,40.003950,89.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1003,116.184239,39.914409,105.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1004,116.171150,39.815128,98.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1005,116.136045,39.742767,109.0,0.0,19.0,61.0,11.16,13.0,False,1
2014-05-01 00:00:00,1006,116.287451,39.987313,87.0,0.0,20.0,56.0,7.92,13.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-03-25 11:00:00,1031,115.988000,40.365000,140.0,0.0,12.0,29.0,3.96,24.0,False,7884
2015-03-25 11:00:00,1032,116.911000,40.499000,111.0,0.0,13.0,30.0,5.04,23.0,False,7884
2015-03-25 11:00:00,1033,117.120000,40.100000,130.0,0.0,13.0,30.0,5.04,23.0,False,7884


,station_id,longitude,latitude,PM25_Concentration,weather,temperature,humidity,wind_speed,wind_direction,filled,delta_t
time,,,,,,,,,,,
2014-05-01 00:00:00,1008,116.348991,39.954047,91.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1012,116.460742,39.937119,78.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1016,116.279082,39.863470,106.0,0.0,20.0,56.0,7.92,13.0,False,1
2014-05-01 00:00:00,1017,116.406155,39.718147,86.0,0.0,19.0,62.0,4.68,13.0,False,1
2014-05-01 00:00:00,1022,116.443448,39.920993,87.0,0.0,20.0,56.0,7.92,13.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-03-25 11:00:00,1024,116.230000,40.216999,119.0,0.0,13.0,29.0,8.50,13.0,False,7884
2015-03-25 11:00:00,1026,117.099999,40.143000,140.0,0.0,13.0,30.0,5.04,23.0,False,7884
2015-03-25 11:00:00,1027,116.628000,40.328000,128.0,0.0,13.0,26.0,7.56,13.0,False,7884


time
2014-05-01 00:00:00     91.0
2014-05-01 00:00:00     78.0
2014-05-01 00:00:00    106.0
2014-05-01 00:00:00     86.0
2014-05-01 00:00:00     87.0
                       ...  
2015-03-25 11:00:00    119.0
2015-03-25 11:00:00    140.0
2015-03-25 11:00:00    128.0
2015-03-25 11:00:00    138.0
2015-03-25 11:00:00    160.0
Name: PM25_Concentration, Length: 78840, dtype: float64

In [33]:
display(train_input.shape, test_input.shape, test_output.shape)

(157680, 11)

(78840, 11)

(78840,)

In [34]:
train_input.reset_index().to_csv('../data/NP/time_feature/fold'+str(fold)+'/train_data_nsgp.csv.gz', index=None)
test_input.reset_index().to_csv('../data/NP/time_feature/fold'+str(fold)+'/test_data_nsgp.csv.gz', index=None)
test_output.reset_index().to_csv('../data/NP/time_feature/fold'+str(fold)+'/test_output_nsgp.csv.gz', index=None)